In [1]:
import os 
import sys 
sys.path.append(os.path.join("..", ".."))
import cv2
import numpy as np 
from utils.imutils import jimshow as show 
from utils.imutils import jimshow_channel as show_channel
import matplotlib.pyplot as plt

import glob
import pandas as pd

Extracting the chosen flower

In [2]:
filepath_f1 = os.path.join("..", "..", "..", "..", "cds-vis-data", "flowers", "image_0001.jpg")
image_f1 = cv2.imread(filepath_f1)
filename_f1 = filepath_f1.split("/")[-1]

In [3]:
hist_f1 = cv2.calcHist([image_f1], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
norm_hist_f1 = cv2.normalize(hist_f1, hist_f1, 0, 1.0, cv2.NORM_MINMAX)

Comparing all other flowers to the extracted flower

In [4]:
filepath_f = os.path.join("..", "..", "..", "..", "cds-vis-data", "flowers")

In [53]:
distances = pd.DataFrame(columns= ("Filename", "Distance"))

for file in sorted(os.listdir(filepath_f)):
    if file != filename_f1:
        filepath_fx = os.path.join(filepath_f, file)
        image_fx = cv2.imread(filepath_fx)
        filename_fx = file.split(".jpg")[0]
        hist_fx = cv2.calcHist([image_fx], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
        norm_hist_fx = cv2.normalize(hist_fx, hist_fx, 0, 1.0, cv2.NORM_MINMAX)

        dist_fx = round(cv2.compareHist(norm_hist_f1, norm_hist_fx, cv2.HISTCMP_CHISQR),5)
        row_fx = [filename_fx, dist_fx]
        
        if len(distances.index)<5:
            distances.loc[len(distances)] = row_fx
                       
        elif ((distances.where(distances["Distance"]>dist_fx))["Distance"]>0).any():
            to_rep = (distances.where(distances["Distance"]>dist_fx))
            to_rep = (to_rep[to_rep["Distance"] == to_rep["Distance"].max()]).values.flatten().tolist()
            distances.replace(to_rep, row_fx)
        else: 
            pass 
    else: 
        pass    

outpath = os.path.join("..", "out", "distances.csv")
distances.to_csv(outpath) 

In [57]:
distances_all = pd.DataFrame(columns= ("Filename", "Distance"))

for file in sorted(os.listdir(filepath_f)):
    if file != filename_f1:
        filepath_fx = os.path.join(filepath_f, file)
        image_fx = cv2.imread(filepath_fx)
        filename_fx = file.split(".jpg")[0]
        hist_fx = cv2.calcHist([image_fx], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
        norm_hist_fx = cv2.normalize(hist_fx, hist_fx, 0, 1.0, cv2.NORM_MINMAX)

        dist_fx = round(cv2.compareHist(norm_hist_f1, norm_hist_fx, cv2.HISTCMP_CHISQR),5)
        row_fx = [filename_fx, dist_fx]

        distances_all.loc[len(distances_all)] = row_fx

    else: 
        pass

In [58]:
distances_all.nsmallest(5, ["Distance"])

,Filename,Distance
926,image_0928,178.12428
874,image_0876,188.54842
771,image_0773,190.08066
140,image_0142,190.20916
1314,image_1316,190.22227
